# HW4A: The Local Model

### Bhaven Patel
### 4/16/2019

I worked with Anthony Rentsch, Lipika Ramaswamy, and Karina Huang on this homework.

My code can be found on my [Github](https://github.com/bhavenp/cs208/blob/master/homework/HW4/HW4_Bhaven_Patel.ipynb).

## Problem 1: Learning Conjunctions in the SQ Model

### (a)

### Centralized Version of SQ Model


For the centralized version of the SQ model, I chose to calculate $p_j = P[x[j]=0 \,\wedge\, y=1]$ for $j=1,...,d$. Then, laplace noise is added to $p_j$ with a scale equal to $\dfrac{GS}{\tilde\epsilon}$, where $GS=\dfrac{1}{n}$ and $\tilde\epsilon= \dfrac{\epsilon}{d}$. Thus, every $p_j$ has a differentially private release $\hat p_j$
$$
\hat p_j = p_j + Lap\left(\dfrac{d}{n\tilde\epsilon}  \right)
$$

Below are the helper functions we generally use.

In [1]:
rm(list=ls())		# Remove any objects in memory
set.seed(123)

# Random draw from Laplace distribution
#
# mu numeric, center of the distribution
# b numeric, spread
# size integer, number of draws
# 
# return Random draws from Laplace distribution
# example:
# 
# rlap(size=1000)

rlap = function(mu=0, b=1, size=1) {
    p <- runif(size) - 0.5
    draws <- mu - b * sgn(p) * log(1 - 2 * abs(p))
    return(draws)
}

# Sign function
# 
# Function to determine what the sign of the passed values should be.
#
# x numeric, value or vector or values
# return The sign of passed values
# example:
#
# sgn(rnorm(10))

sgn <- function(x) {
    return(ifelse(x < 0, -1, 1))
}

In [2]:
##function to create the matrix that holds an indicator if x_j==0 & y==1
createConjunctionMat <- function(xData, yData){
    #create matrix to hold indicator if x_j==0 & y==1
    result_matrix = matrix(0, nrow=nrow(xData), ncol=ncol(xData));
    for(i in 1:nrow(xData)){
        if(yData[i] == 1){ #only need to consider row if y=1
            result_matrix[i, ] <- (xData[i, ] == 0); #check if x_j == 0
        }
    }
    return(result_matrix);
}

#function to calculate DP-releases for each probability
probRelease <- function(xMat, epsilon=1.0){
    probs <- colMeans(xMat); #calculate true probabilities
    sensitivity <- 1 / nrow(xMat); #sensitivity is 1/n
    scale <- sensitivity / epsilon;
    dpProbs <- probs + rlap(mu=0, b=scale, size=length(probs)); #add laplace noise to the true probabilities

	return(list(release=dpProbs, true=probs) );
}

#function that ties together the different parts for doing a DP release of the probabilities for a 
## xData: matrix of {0,1}
## yData: vector of {0,1}, same length as number of rows in xData
## epsilon: total privacy-loss parameter. This will get split up by the number of columns in xData that 
##          we must release probabilities for
## returns a vector of the indices corresponding to the columns of xData that predict yData well
centrlDP_SQAlg <- function(xData, yData, totEpsilon=1.0, threshold=1e-4){
    pMatrix <- createConjunctionMat(xData=xData, yData=yData); #create conjunction matrix
#     print(totEpsilon/ncol(xData))
    dpRelease <- probRelease(pMatrix, epsilon = totEpsilon/ncol(xData) ); #get DP release of probabilities
#     cat(dpRelease$true);
#     cat(dpRelease$release);
    indices <- which(dpRelease$release < threshold); #get indices with probability less than threshold
    return(indices);
}

In [13]:
# pMatrix <- createConjunctionMat(xData=mydata[,0:10], yData=mydata[['y']])
# pMatrix[1:5, ]*5
# dpRelease <- probRelease(pMatrix, epsilon = 0.5);
# dpRelease$true
# dpRelease$release

#read in the test data
# mydata <- read.csv('../../data/hw4testdata.csv');
# mydata[0:10, ]

#get set of features to use as predictors
# centrlDP_SQAlg(xData = mydata[, 1:10], yData = mydata[['y']], totEpsilon = 0.5)

0,0,0,5,5,0,0,0,0,5
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0


### Local Model

#### describe local model implementation

In [3]:
#local release mechanism works for x as a scalar
localRelease <- function(x, values=c(0,1), epsilon){
	draw <- runif(n=1, min=0, max=1);
	cutoff <- 1/(1+exp(epsilon));
    if(draw < cutoff){ #we are going to flip the our value
        return(values[ !values %in% x]); #create flag with (!values %in% x) for opposite value of 'x'
    }else{ #draw > cutoff so we return true value
        return(x);
    }
}


#function to apply correction for our 0/1 values after local release
correction <- function(releaseMat, epsilon){
	inflation <- (exp(epsilon) + 1)/(exp(epsilon) - 1); #coefficient to multiply each value in the release by
    additive <- -1 / (exp(epsilon) - 1); #additive factor to apply for because data is {0,1}
    cat(inflation, additive);
	correctedMat <- releaseMat*inflation + additive;
	return(correctedMat);
}


#function that ties together the different parts for doing a DP release of the probabilities for a local model
## xData: matrix of {0,1}
## yData: vector of {0,1}, same length as number of rows in xData
## epsilon: total privacy-loss parameter. This will get split up by the number of columns in xData that 
##          we must release probabilities for
## returns a vector of the indices corresponding to the columns of xData that predict yData well
localDP_SQAlg <- function(xData, yData, totEpsilon=1.0, threshold=1e-4){
    cat(paste(Sys.time(),"\n") )
    pMatrix <- createConjunctionMat(xData=xData, yData=yData); #create conjunction matrix
    epsSplit <- totEpsilon / ncol(xData); #divide the total epsilon by the number of features
    #perform local release of conjunction matrix
    
    for(r in 1:nrow(pMatrix)){
#         cat(paste(r, Sys.time(), "\n") );
        for(c in 1:ncol(pMatrix)){
            pMatrix[r, c] <- localRelease(x=pMatrix[r, c], values = c(0,1), epsilon = epsSplit);
        }
    }
    cat(paste(Sys.time(),"\n") )
#     #perform correction
#     correctedMat <- correction(releaseMat = pMatrix, epsilon = epsSplit);
#     dpRelease <- colMeans(correctedMat); #calculate the probabilities for each feature
    
#     indices <- which(dpRelease < threshold); #get indices with probability less than threshold
#     return(indices);
}

In [4]:
mydata <- read.csv('../../data/hw4testdata.csv');
localDP_SQAlg(xData = mydata[, 1:10], yData = mydata[['y']], totEpsilon = 0.5)

2019-04-14 23:29:22 
2019-04-14 23:33:31 


### (b)

### Centralized DP algorithm

I will refer to the $d$ columns in each dataset $x_i$ as features.<br>
$P[\hat S \supseteq S]$ is the probability that set $S$ contains a feature $j$ that $\hat S$ does not. This equivalent to determining the probability that at least one of the features $j \in S$ has DP-released $\hat p_j$ that is greater than the threshold $t$ we specify. Thus, we get
$$
P[\hat S \supseteq S] = \sum_{j\in S} P[\hat p_j > t]
$$

For the centralized DP SQ algorithm, I chose $\hat p_j = p_j + Lap\left(\dfrac{d}{n\tilde\epsilon}\right)$. Additionally, we know that for each $j \in S$, we have $p_j =0$. Thus, we get
$$\begin{align}
P[\hat S \supseteq S] &= \sum_{j\in S} P[p_j + Lap\left(\dfrac{d}{n\tilde\epsilon}\right) > t]\\
&= \sum_{j\in S} P[0 + Lap\left(\dfrac{d}{n\tilde\epsilon}\right) > t]
\end{align}$$

This can be equation can be expanded to be in terms of $t$, $n$, $\epsilon$, and $|S|$ because we can expand the probability that a value chosen from the Laplace distribution is greater than $t$ and $\sum_{j\in S}=|S|$:
$$\begin{align}
&= \sum_{j\in S} P[0 + Lap\left(\dfrac{d}{n\tilde\epsilon}\right) > t] \\
&= |S| \cdot \int_t^{\infty} \dfrac{e^{(-|y|\cdot n\epsilon / d)}\cdot n\epsilon}{2d} dy\\
\end{align}$$

Now, if we say that $P[\hat S \supseteq S] \leq 0.1$, we can find a lower bound on what the threshold $t$ should be:
$$\begin{align}
P[\hat S \supseteq S] &\leq 0.1\\
\\
|S| \cdot \int_t^{\infty} \dfrac{e^{(-|y|\cdot n\epsilon / d)}\cdot n\epsilon}{2d} dy &\leq 0.1\\
\\
|S|\dfrac{n\epsilon}{2d} \cdot \int_t^{\infty} e^{(-|y|\cdot n\epsilon / d)} dy &\leq 0.1\\
\\
\\
\int_t^{\infty} e^{(-|y|\cdot n\epsilon / d)} dy &\leq \dfrac{(0.1)\cdot 2d}{|S|n\epsilon}\\
\\
\left[\dfrac{-d}{n\epsilon} e^{(-y\cdot n\epsilon / d)} \right]_t^{\infty} &\leq \dfrac{(0.1)\cdot 2d}{|S|n\epsilon}\\
\\
\dfrac{-d}{n\epsilon} \left[ e^{(-\infty \cdot n\epsilon / d)} - e^{(-t \cdot n\epsilon / d)} \right] &\leq \dfrac{(0.1)\cdot 2d}{|S|n\epsilon}\\
\\
-1 \left[ 0 - e^{(-t \cdot n\epsilon / d)} \right] &\leq \dfrac{(0.2)}{|S|}\\
\\
e^{(-t \cdot n\epsilon / d)} &\leq \dfrac{(0.2)}{|S|}\\
\\
-t  &\leq \dfrac{d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)\\
\\
t  &\leq \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)\\
\end{align}$$

Thus, for the centralized DP SQ algorithm, we find that $t  \leq \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)$ if we want $P[\hat S \supseteq S] \leq 0.1$.

### (c)

In [48]:
#read in the test data
mydata <- read.csv('../../data/CaPUMS5full.csv');
mydata[0:10, ]

#get set of features to use as predictors
attrInd <- centrlDP_SQAlg(xData = mydata[, 1:10], yData = mydata[['targetted']], totEpsilon = 1, threshold = 1e-4);
cat(attrInd);

sex,married,black,asian,collegedegree,employed,militaryservice,uscitizen,disability,englishability,blackfemale,targetted
1,1,0,0,0,1,0,1,0,1,0,0
1,1,0,0,0,0,0,1,0,1,0,0
0,1,0,0,0,0,1,1,0,1,0,0
0,0,0,0,0,0,0,1,1,1,0,0
0,1,0,0,0,0,0,1,0,1,0,0
1,1,0,0,0,0,0,1,0,1,0,0
0,1,0,0,0,1,0,1,0,1,0,1
1,1,0,0,0,0,0,1,0,1,0,0
0,1,0,0,0,1,0,1,0,1,0,1
1,1,0,0,0,1,0,1,0,1,0,0


In [49]:
mydata <- read.csv('../../data/CaPUMS5full.csv');
nrow(mydata)

[1] 1223992